# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 10:47AM,238980,21,399290112,"NBTY Global, Inc.",Released
1,Jun 7 2022 10:44AM,238979,19,ACG-2102490260,ACG North America LLC,Released
2,Jun 7 2022 10:42AM,238978,20,ELSE-8113129,Else Nutrition,Released
3,Jun 7 2022 10:32AM,238977,10,DNMD-21715,Emerginnova,Released
4,Jun 7 2022 10:30AM,238976,21,399290070,"NBTY Global, Inc.",Released
5,Jun 7 2022 10:27AM,238942,20,7994551,"ACI Healthcare USA, Inc.",Released
6,Jun 7 2022 10:27AM,238942,20,ACI-7942967-BO,"ACI Healthcare USA, Inc.",Released
7,Jun 7 2022 10:27AM,238942,20,ACI-8091745,"ACI Healthcare USA, Inc.",Released
8,Jun 7 2022 10:27AM,238942,20,ACI-8091749,"ACI Healthcare USA, Inc.",Released
9,Jun 7 2022 10:27AM,238942,20,8096314,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,238976,Released,1
43,238977,Released,1
44,238978,Released,1
45,238979,Released,1
46,238980,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238976,NaN,1.0
238977,NaN,1.0
238978,NaN,1.0
238979,NaN,1.0
238980,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238868,0.0,1.0
238873,0.0,1.0
238879,36.0,8.0
238883,0.0,1.0
238888,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238868,0,1
238873,0,1
238879,36,8
238883,0,1
238888,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238868,0,1
1,238873,0,1
2,238879,36,8
3,238883,0,1
4,238888,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238868,,1
1,238873,,1
2,238879,36,8
3,238883,,1
4,238888,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 10:47AM,238980,21,"NBTY Global, Inc."
1,Jun 7 2022 10:44AM,238979,19,ACG North America LLC
2,Jun 7 2022 10:42AM,238978,20,Else Nutrition
3,Jun 7 2022 10:32AM,238977,10,Emerginnova
4,Jun 7 2022 10:30AM,238976,21,"NBTY Global, Inc."
5,Jun 7 2022 10:27AM,238942,20,"ACI Healthcare USA, Inc."
22,Jun 7 2022 10:24AM,238975,10,Emerginnova
23,Jun 7 2022 10:20AM,238926,20,"ACI Healthcare USA, Inc."
24,Jun 7 2022 10:15AM,238974,21,"NBTY Global, Inc."
25,Jun 7 2022 10:12AM,238973,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 7 2022 10:47AM,238980,21,"NBTY Global, Inc.",,1
1,Jun 7 2022 10:44AM,238979,19,ACG North America LLC,,1
2,Jun 7 2022 10:42AM,238978,20,Else Nutrition,,1
3,Jun 7 2022 10:32AM,238977,10,Emerginnova,,1
4,Jun 7 2022 10:30AM,238976,21,"NBTY Global, Inc.",,1
5,Jun 7 2022 10:27AM,238942,20,"ACI Healthcare USA, Inc.",,17
6,Jun 7 2022 10:24AM,238975,10,Emerginnova,,1
7,Jun 7 2022 10:20AM,238926,20,"ACI Healthcare USA, Inc.",,1
8,Jun 7 2022 10:15AM,238974,21,"NBTY Global, Inc.",,1
9,Jun 7 2022 10:12AM,238973,10,"CLINUVEL, Inc.",,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 10:47AM,238980,21,"NBTY Global, Inc.",1,
1,Jun 7 2022 10:44AM,238979,19,ACG North America LLC,1,
2,Jun 7 2022 10:42AM,238978,20,Else Nutrition,1,
3,Jun 7 2022 10:32AM,238977,10,Emerginnova,1,
4,Jun 7 2022 10:30AM,238976,21,"NBTY Global, Inc.",1,
5,Jun 7 2022 10:27AM,238942,20,"ACI Healthcare USA, Inc.",17,
6,Jun 7 2022 10:24AM,238975,10,Emerginnova,1,
7,Jun 7 2022 10:20AM,238926,20,"ACI Healthcare USA, Inc.",1,
8,Jun 7 2022 10:15AM,238974,21,"NBTY Global, Inc.",1,
9,Jun 7 2022 10:12AM,238973,10,"CLINUVEL, Inc.",3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 10:47AM,238980,21,"NBTY Global, Inc.",1,
1,Jun 7 2022 10:44AM,238979,19,ACG North America LLC,1,
2,Jun 7 2022 10:42AM,238978,20,Else Nutrition,1,
3,Jun 7 2022 10:32AM,238977,10,Emerginnova,1,
4,Jun 7 2022 10:30AM,238976,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 10:47AM,238980,21,"NBTY Global, Inc.",1.0,NaN
1,Jun 7 2022 10:44AM,238979,19,ACG North America LLC,1.0,NaN
2,Jun 7 2022 10:42AM,238978,20,Else Nutrition,1.0,NaN
3,Jun 7 2022 10:32AM,238977,10,Emerginnova,1.0,NaN
4,Jun 7 2022 10:30AM,238976,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 10:47AM,238980,21,"NBTY Global, Inc.",1.0,0.0
1,Jun 7 2022 10:44AM,238979,19,ACG North America LLC,1.0,0.0
2,Jun 7 2022 10:42AM,238978,20,Else Nutrition,1.0,0.0
3,Jun 7 2022 10:32AM,238977,10,Emerginnova,1.0,0.0
4,Jun 7 2022 10:30AM,238976,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2867445,32.0,6.0
15,955764,83.0,0.0
16,477814,2.0,24.0
17,477784,2.0,0.0
18,716733,3.0,0.0
19,1433560,6.0,1.0
20,2150359,82.0,82.0
21,1194776,5.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2867445,32.0,6.0
1,15,955764,83.0,0.0
2,16,477814,2.0,24.0
3,17,477784,2.0,0.0
4,18,716733,3.0,0.0
5,19,1433560,6.0,1.0
6,20,2150359,82.0,82.0
7,21,1194776,5.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,32.0,6.0
1,15,83.0,0.0
2,16,2.0,24.0
3,17,2.0,0.0
4,18,3.0,0.0
5,19,6.0,1.0
6,20,82.0,82.0
7,21,5.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,15,Released,83.0
2,16,Released,2.0
3,17,Released,2.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21
Status,,,,,,,,
Executing,6.0,0.0,24.0,0.0,0.0,1.0,82.0,0.0
Released,32.0,83.0,2.0,2.0,3.0,6.0,82.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21
0,Executing,6.0,0.0,24.0,0.0,0.0,1.0,82.0,0.0
1,Released,32.0,83.0,2.0,2.0,3.0,6.0,82.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21
0,Executing,6.0,0.0,24.0,0.0,0.0,1.0,82.0,0.0
1,Released,32.0,83.0,2.0,2.0,3.0,6.0,82.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()